# Get repsonse from an LLM


To get started, [get an API key](https://g.co/ai/idxGetGeminiKey) and replace the word `API KEY` below with your API key:

In [33]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
api_key = os.getenv("SECRET_KEY")

# Ensure the API key is not None
if not api_key:
    raise ValueError("API key not found. Please set SECRET_KEY in the .env file.")

# Configure the Generative AI library
genai.configure(api_key=api_key)

# Define the prompt, query, and observation
prompt = "You are a helpful assistant."
query = "What is the weather today?"
observation = "The user is asking about the current weather."

# Combine the system prompt and user query
full_prompt = f"{prompt}\n\nUser: {query}\nAssistant: {observation}\n\nUser:"

# Create a model and generate content
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(full_prompt)

# Print the response
if response and response.parts:
    print(response.parts[0].text)
else:
    print("No response received.")



Okay, I'm ready for the next user input.



Build your own Agent 

In [35]:
import re

# Define your prompt and query
prompt = """ 
You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions.
1. Thought: Describe your thoughts about the question you have been asked.
2. Action: run one of the actions available to you - then return PAUSE.
3. PAUSE
4. Observation: will be the result of running those actions.

Available actions:
- getCurrentWeather: 
    E.g. getCurrentWeather: Salt Lake City
    Returns the current weather of the location specified.
- getLocation:
    E.g. getLocation: null
    Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: getLocation: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: getCurrentWeather: New York City
PAUSE

You'll then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding areas.>
"""

query = "What is the current weather at my location?"

# Dummy functions to simulate API calls
def getLocation(place=None):
    return {"city": "New York City", "state": "NY", "country": "USA"}

def getCurrentWeather(place=None):
    return {"temperature": "2", "unit": "F", "forecast": "snowy"}

# Map available functions
available_functions = {
    "getLocation": getLocation,
    "getCurrentWeather": getCurrentWeather
}

# Simulate an LLM response generator (Replace this with your LLM integration)
def call_llm(current_prompt):
    # Simulated LLM response to match the defined actions
    if "getLocation" in current_prompt:
        return "Observation: New York City, NY"
    elif "getCurrentWeather" in current_prompt:
        return "Observation: { location: 'New York City, NY', forecast: ['snowy'] }"
    else:
        return "No valid action found."

# Regex for extracting actions
action_regex = r"^Action: (\w+): (.*)$"
max_calls = 3

# Initial prompt to start the loop
current_prompt = f"{prompt}\n\nQuestion: {query}\nThought:"

for i in range(max_calls):
    print(f"Call {i + 1}:")

    # Simulate LLM response
    response = call_llm(current_prompt)

    # Check if response contains an observation
    if response.startswith("Observation:"):
        observation = response.split("Observation:")[1].strip()
        print("Observation:", observation)

        # Update the prompt with the observation
        current_prompt += f"\nObservation: {observation}\nThought:"
    else:
        # Try to find and execute an action
        response_lines = response.strip().split("\n")
        found_action_str = next((line for line in response_lines if re.match(action_regex, line)), None)

        if found_action_str:
            actions = re.match(action_regex, found_action_str)
            if actions:
                action, action_arg = actions.groups()
                if action in available_functions:
                    observation = available_functions[action](action_arg)
                    print("Observation:", observation)

                    # Update the prompt with the action and observation
                    current_prompt += f"\nAction: {action}: {action_arg}\nObservation: {observation}\nThought:"
                else:
                    print(f"Action '{action}' is not available.")
            else:
                print("No valid action found.")
        else:
            print("No valid action found.")

# Final answer generation
#print("Final Answer:", "Based on the observations, the weather is snowy in New York City.")


Call 1:
Observation: New York City, NY
Call 2:
Observation: New York City, NY
Call 3:
Observation: New York City, NY
